In [1]:
%load_ext autoreload
%autoreload 2

In [460]:
import pandas as pd
import jax.numpy as jnp
import glob
import data_loading
import eval

In [461]:
# load sample lobster data
message_paths = glob.glob('data/*message*.csv')
book_paths = glob.glob('data/*orderbook*.csv')

In [462]:
m = data_loading.load_message_df(message_paths[0])
b = data_loading.load_book_df(book_paths[0])

In [9]:
m

,time,event_type,order_id,size,price,direction
0,34200.015105074,4,15818974,4,5794000,1
1,34200.05990197,4,15818974,300,5794000,1
2,34200.113246707,5,0,1,5795100,1
3,34200.113246707,5,0,1,5795000,1
4,34200.113246707,5,0,1,5794900,1
...,...,...,...,...,...,...
147911,57599.544131783,1,287157290,300,5655600,-1
147912,57599.544132627,1,287157291,200,5655600,-1
147913,57599.544133264,1,287157292,300,5655600,-1
147914,57599.582154048,3,287027555,100,5649700,1


In [72]:
eval.time_to_first_fill(m).mean()

17.77700116925748

In [71]:
eval.time_to_cancel(m).mean()

7.965471268527362

In [97]:
b

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147911,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147912,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2400,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147913,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [80]:
eval.l1_volume(m, b)

,ask_vol,bid_vol,time_weight
0,100,496,NaN
1,100,196,0.000001914408993084087627622518347
2,100,196,0.000002279703581393350855837805738
3,100,196,0
4,100,196,0
...,...,...,...
147911,42,200,8.290649081095105334056372105E-11
147912,42,200,3.606859703321788093785349514E-11
147913,42,200,2.722238899308032009172118057E-11
147914,42,200,0.000001624829783155234450326830761


In [96]:
eval.total_volume(m, b, 1)

,ask_vol,bid_vol,time_weight
0,100,5802300,NaN
1,100,5802300,0.000001914408993084087627622518347
2,100,5802300,0.000002279703581393350855837805738
3,100,5802300,0
4,100,5802300,0
...,...,...,...
147911,42,5652200,8.290649081095105334056372105E-11
147912,42,5652200,3.606859703321788093785349514E-11
147913,42,5652200,2.722238899308032009172118057E-11
147914,42,5652200,0.000001624829783155234450326830761


In [115]:
eval.limit_order_depth(m, b)

(10        2450.0
 12        3050.0
 14        3350.0
 16        3550.0
 18        3550.0
            ...  
 147909    3900.0
 147910    3900.0
 147911    3900.0
 147912    3900.0
 147913    3900.0
 Length: 37136, dtype: float64,
 7         4150.0
 8         6950.0
 9         8150.0
 11        6450.0
 13        6550.0
            ...  
 147889    3950.0
 147891    2600.0
 147892    3700.0
 147894    3200.0
 147896    4000.0
 Length: 34122, dtype: float64)

In [116]:
eval.cancellation_depth(m, b)

(45         9250.0
 47        11650.0
 76         3050.0
 77         2750.0
 81         3050.0
            ...   
 147876     2850.0
 147897     3300.0
 147898     2200.0
 147904     3900.0
 147915     3300.0
 Length: 33222, dtype: float64,
 44        25650.0
 46        23250.0
 69         7550.0
 70        14550.0
 71         9550.0
            ...   
 147883     3800.0
 147887     4000.0
 147893     2600.0
 147903     1100.0
 147914     2000.0
 Length: 31724, dtype: float64)

In [125]:
eval._order_levels(m, b, (1,2,3))

(3          1
 39         1
 42         1
 43         1
 47         1
           ..
 136005    10
 136019    10
 136025    10
 136113    10
 136217    10
 Length: 49808, dtype: int64,
 0          1
 40         1
 44         1
 64         1
 65         1
           ..
 135877    10
 136010    10
 136030    10
 136053    10
 136194    10
 Length: 45838, dtype: int64)

In [127]:
eval.cancel_order_levels(m, b)

(8         1
 10        1
 12        1
 47        1
 52        1
          ..
 63101    10
 63371    10
 63406    10
 63420    10
 64446    10
 Length: 12672, dtype: int64,
 73        1
 77        1
 108       1
 124       1
 352       1
          ..
 63942    10
 64079    10
 64143    10
 64264    10
 64560    10
 Length: 11716, dtype: int64)

In [129]:
b

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147911,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147912,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2400,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147913,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [128]:
eval.spread(b)

0         8300
1         8300
2         8300
3         8300
4         8300
          ... 
147911    1000
147912    1000
147913    1000
147914    1000
147915    1000
Length: 147916, dtype: int64

In [188]:
ret = eval.mid_returns(m, b, '1min')
ret

time
0 days 09:31:00         NaN
0 days 09:32:00    0.000701
0 days 09:33:00   -0.001393
0 days 09:34:00    0.000329
0 days 09:35:00    0.000251
                     ...   
0 days 15:56:00    0.000390
0 days 15:57:00    0.000425
0 days 15:58:00   -0.000354
0 days 15:59:00    0.001275
0 days 16:00:00    0.000416
Length: 390, dtype: float64

In [189]:
eval.time_to_first_fill(m)

order_id
858821       12.002251
2175511      12.002251
2176461      12.002898
4496151      11.939107
4773351      11.969646
               ...    
287076073     0.102907
287081956     0.000316
287081969     0.000234
287091827     0.275303
287107097     0.114062
Length: 5480, dtype: float64

In [174]:
eval.autocorr(ret)

array([-0.13855764])

In [182]:
eval.autocorr(ret**2, 10, alpha=None)

array([ 1.00000000e+00,  3.97760808e-01,  1.46716661e-01,  1.21313544e-01,
        3.83928983e-02, -1.50425941e-02,  6.71807872e-02,  1.02656084e-01,
        6.61179836e-02,  3.98363877e-04, -2.46290823e-02])

In [183]:
eval.l1_volume(m, b)

,ask_vol,bid_vol,time_weight
0,100,496,NaN
1,100,196,0.000001914408993084087627622518347
2,100,196,0.000002279703581393350855837805738
3,100,196,0
4,100,196,0
...,...,...,...
147911,42,200,8.290649081095105334056372105E-11
147912,42,200,3.606859703321788093785349514E-11
147913,42,200,2.722238899308032009172118057E-11
147914,42,200,0.000001624829783155234450326830761


In [194]:
eval.total_volume(m, b, 2)

,ask_vol,bid_vol,time_weight
time,,,
0 days 09:30:00.015105074,596,11596300,NaN
0 days 09:30:00.059901970,296,11596300,0.000001914408993084087627622518347
0 days 09:30:00.113246707,296,11596300,0.000002279703581393350855837805738
0 days 09:30:00.113246707,296,11596300,0
0 days 09:30:00.113246707,296,11596300,0
...,...,...,...
0 days 15:59:59.544131783,242,11303400,8.290649081095105334056372105E-11
0 days 15:59:59.544132627,242,11303400,3.606859703321788093785349514E-11
0 days 15:59:59.544133264,242,11303400,2.722238899308032009172118057E-11


In [ ]:
# TODO: how to calculate by time?
#       should be able to do this by calculating on whole day data and then grouping
#       or by calculating for shorter time periods and then grouping and aggregating

In [490]:
loader = data_loading.Simple_Loader('data_test_real', 'data_test_gen')

In [574]:
eval.limit_order_depth(m_real, b_real)

(time
 2012-06-21 09:30:00.202322801    2450.0
 2012-06-21 09:30:00.202322801    3050.0
 2012-06-21 09:30:00.202322801    3350.0
 2012-06-21 09:30:00.202322801    3550.0
 2012-06-21 09:30:00.202322801    3550.0
                                   ...  
 2012-06-21 15:59:59.544126313    3900.0
 2012-06-21 15:59:59.544129843    3900.0
 2012-06-21 15:59:59.544131783    3900.0
 2012-06-21 15:59:59.544132627    3900.0
 2012-06-21 15:59:59.544133264    3900.0
 Length: 37136, dtype: float64,
 time
 2012-06-21 09:30:00.154178213    4150.0
 2012-06-21 09:30:00.173594288    6950.0
 2012-06-21 09:30:00.202322801    8150.0
 2012-06-21 09:30:00.202322801    6450.0
 2012-06-21 09:30:00.202322801    6550.0
                                   ...  
 2012-06-21 15:59:58.844529892    3950.0
 2012-06-21 15:59:58.958997603    2600.0
 2012-06-21 15:59:59.009636180    3700.0
 2012-06-21 15:59:59.122827419    3200.0
 2012-06-21 15:59:59.146213451    4000.0
 Length: 34122, dtype: float64)

In [ ]:
### FIRST TODO: think about metric conditioning! (relevant for stylized facts here but also for distributional metrics)
#               i.e. find a principled way to condition not only on time but other factors in the input data?
#               (--> need to pass input data used to condition generation to metric functions?)
#               or can we conditon only on part of the realized data? e.g. 

In [ ]:
# think about final output format --> how to handle multiple generations of the same time period?
# where to take means, where to evaluate distributions, etc.

# which divergence metrics to use to compare real and generated distributions?

In [580]:
# currently: one battery of metrics for a single data series (real or generated)
# could do (sort of) unconditional comparison (only conditonal on time of day): calc metrics for time of day and compare these distributions

eval.compute_metrics(loader)

,spread,mid_returns_1min,ask_vol,bid_vol,time_weight,ask_vol_10,bid_vol_10,time_weight,ask_limit_depth,bid_limit_depth,ask_cancel_depth,bid_cancel_depth,ask_limit_level,bid_limit_level,ask_cancel_level,bid_cancel_level
time,,,,,,,,,,,,,,,,
2012-06-21 09:35:00,8376.626367,-0.000121,241.099021,190.697179,0.000007,1498.368451,5.782912e+07,0.000007,5917.887029,6669.811321,6383.293269,6753.533569,2.035565,2.333333,2.160839,2.637037
2012-06-21 09:40:00,5430.894495,-0.000610,159.560780,134.629817,0.000003,1240.393349,5.767007e+07,0.000003,6813.548387,3269.740475,7475.000000,3095.104108,3.077419,1.569851,3.455939,1.959437
2012-06-21 09:45:00,5097.472119,0.000010,101.460967,103.371747,0.000010,895.565056,5.758533e+07,0.000010,5591.634981,3803.703704,5707.287449,3898.204420,3.965779,2.751323,4.217573,3.032432
2012-06-21 09:50:00,4264.502470,0.000328,132.516584,144.983063,0.000009,1383.552576,5.765170e+07,0.000009,5028.459821,4146.800000,4946.596244,4200.896861,3.339286,3.388000,3.305361,3.590909
2012-06-21 09:55:00,3642.405063,-0.000142,133.750730,104.103213,0.000006,852.582278,5.764077e+07,0.000006,3340.394089,2966.084788,3324.000000,2974.181360,2.878489,3.049875,3.300172,3.650386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-06-21 15:45:00,2284.818805,-0.000117,177.146915,200.842311,0.000006,1150.606268,5.649014e+07,0.000006,2060.957447,2310.307018,2381.923077,2404.973822,3.380851,3.789474,4.398458,4.333333
2012-06-21 15:50:00,2314.896266,-0.000110,126.977593,265.901660,0.000005,1337.497510,5.646090e+07,0.000005,2150.838415,1761.864407,2495.808967,2041.184971,3.042683,2.803874,3.661538,3.661765
2012-06-21 15:55:00,1574.562018,-0.000285,224.134198,154.881570,0.000005,1303.623686,5.643338e+07,0.000005,1360.918675,1492.100840,1739.768340,1702.589641,2.697289,3.292437,3.778420,3.908730


In [491]:
m_real, b_real, m_gen, b_gen = loader[0]

In [396]:
for i in loader:
    print(i)

(                                time  event_type   order_id  size    price  \
0      2012-06-21 09:30:00.015105074           4   15818974     4  5794000   
1      2012-06-21 09:30:00.005990197           4   15818974   300  5794000   
2      2012-06-21 09:30:00.113246707           5          0     1  5795100   
3      2012-06-21 09:30:00.113246707           5          0     1  5795000   
4      2012-06-21 09:30:00.113246707           5          0     1  5794900   
...                              ...         ...        ...   ...      ...   
147911 2012-06-21 15:59:59.544131783           1  287157290   300  5655600   
147912 2012-06-21 15:59:59.544132627           1  287157291   200  5655600   
147913 2012-06-21 15:59:59.544133264           1  287157292   300  5655600   
147914 2012-06-21 15:59:59.582154048           3  287027555   100  5649700   
147915 2012-06-21 15:59:59.871751084           3  287151822   100  5655000   

        direction  
0               1  
1               1  
2 

In [ ]:
ask_lvls, bid_lvls = eval.limit_order_levels(m_real, b_real)

order_prices (71258,)
level_prices (71258, 20)
(71258, 2)


In [ ]:
ask_lvls

time
2012-06-21 09:30:00.202322801    1
2012-06-21 09:30:00.202322801    2
2012-06-21 09:30:00.202322801    3
2012-06-21 09:30:00.202322801    4
2012-06-21 09:30:00.202322801    4
                                ..
2012-06-21 15:59:59.544126313    3
2012-06-21 15:59:59.544129843    3
2012-06-21 15:59:59.544131783    3
2012-06-21 15:59:59.544132627    3
2012-06-21 15:59:59.544133264    3
Length: 37136, dtype: int64

In [476]:
m_real

,time,event_type,order_id,size,price,direction
0,34200.015105074,4,15818974,4,5794000,1
1,34200.05990197,4,15818974,300,5794000,1
2,34200.113246707,5,0,1,5795100,1
3,34200.113246707,5,0,1,5795000,1
4,34200.113246707,5,0,1,5794900,1
...,...,...,...,...,...,...
147911,57599.544131783,1,287157290,300,5655600,-1
147912,57599.544132627,1,287157291,200,5655600,-1
147913,57599.544133264,1,287157292,300,5655600,-1
147914,57599.582154048,3,287027555,100,5649700,1


In [466]:
b_real

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147911,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147912,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2400,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147913,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [527]:
eval.mid_returns(m_real, b_real)

time
2012-06-21 09:31:00         NaN
2012-06-21 09:32:00    0.000701
2012-06-21 09:33:00   -0.001393
2012-06-21 09:34:00    0.000329
2012-06-21 09:35:00    0.000251
                         ...   
2012-06-21 15:56:00    0.000390
2012-06-21 15:57:00    0.000425
2012-06-21 15:58:00   -0.000354
2012-06-21 15:59:00    0.001275
2012-06-21 16:00:00    0.000416
Freq: min, Length: 390, dtype: float64

In [534]:
m_real.iloc[1163]

time          2012-06-21 09:31:38.384531913
event_type                                1
order_id                           19519131
size                                    100
price                               5809400
direction                                -1
Name: 1163, dtype: object

In [533]:
b_real.iloc[1163]

0     5795900
1         500
2     5775600
3         200
4     5798900
5          25
6     5775500
7         100
8     5799300
9           2
10    5775400
11        100
12    5799500
13         50
14    5775000
15        400
16    5799800
17          5
18    5773000
19        100
20    5800000
21       2937
22    5770000
23          7
24    5800100
25         10
26    5767500
27        100
28    5801000
29         10
30    5766000
31        100
32    5805000
33        100
34    5765000
35          5
36    5809400
37        100
38    5762500
39          1
Name: 1163, dtype: int64

In [477]:
m_real.loc[m_real.event_type == 3].price.max()

5809400

In [535]:
b_real.loc[m_real.loc[m_real.event_type == 3].index].max()

0     5798900
1        2700
2     5791200
3        2191
4     5799500
5        1500
6     5790000
7        2101
8     5799800
9        2700
10    5789900
11       2320
12    5800000
13       2537
14    5787000
15       2320
16    5800100
17       2937
18    5785000
19       2320
20    5801000
21       2937
22    5782100
23       2320
24    5802300
25       2937
26    5780000
27       2410
28    5805000
29       2937
30    5779900
31       2310
32    5806300
33       2937
34    5774500
35       2410
36    5809600
37       2945
38    5772400
39       2410
dtype: int64

In [ ]:
# ISSUE: some delete (type 3) messages are not in the book data --> outside levels but still in messages???

In [510]:
eval.autocorr(eval.mid_returns(m_real, b_real, '1min'), 4)
# eval.mid_returns(m_real, b_real, '5min')

array([ 1.        , -0.13855764, -0.02199417,  0.00926521,  0.00570809])

In [432]:
b_real.loc[m_real.loc[m_real.event_type == 3].index]

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
44,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5805000,100,5771800,100,5806300,100,5766000,100
45,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5806300,100,5771800,100,5806700,100,5766000,100
46,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5806300,100,5766000,100,5806700,100,5762600,100
47,5798900,25,5791200,100,5799500,50,5790000,41,5799800,5,...,5779900,1,5806300,100,5766000,100,5809600,50,5762600,100
69,5797900,210,5791200,100,5798900,25,5790000,41,5799500,50,...,5775100,110,5805000,100,5774500,3,5806300,100,5772400,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147898,5655600,200,5651200,200,5655700,100,5651100,100,5656000,100,...,5649600,100,5657700,800,5649400,100,5657800,100,5649200,300
147903,5652200,42,5651200,200,5655000,200,5651100,100,5655600,200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147904,5652200,42,5651200,200,5655000,200,5651100,100,5655600,100,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [536]:
order_prices = m_real[m_real.event_type.isin((3,))]
level_prices = b_real.loc[order_prices.index, 0::2]
order_prices.index = order_prices.time
order_prices = order_prices.price

In [551]:
order_prices.iloc[0]

5769400

In [553]:
level_prices.iloc[0]

0     5798900
2     5791200
4     5799500
6     5790000
8     5799800
10    5789900
12    5800000
14    5787000
16    5800100
18    5785000
20    5801000
22    5782100
24    5802300
26    5780000
28    5804300
30    5779900
32    5805000
34    5771800
36    5806300
38    5766000
Name: 44, dtype: int64

In [549]:
(order_prices.values == level_prices.values.T).T[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [537]:
import numpy as np

lvl_idx = np.argwhere((order_prices.values == level_prices.values.T).T)

In [539]:
lvl_idx.shape

(24369, 2)

In [565]:
eval._order_levels(m_real, b_real, (2,3))

(time
 2012-06-21 09:30:00.205119161     8
 2012-06-21 09:30:00.205183697    10
 2012-06-21 09:30:03.650345438     1
 2012-06-21 09:30:03.650507021     1
 2012-06-21 09:30:04.414627059     1
                                  ..
 2012-06-21 15:59:58.304148694     6
 2012-06-21 15:59:59.148979716     6
 2012-06-21 15:59:59.320522524     1
 2012-06-21 15:59:59.489629981     3
 2012-06-21 15:59:59.871751084     2
 Length: 33237, dtype: int64,
 time
 2012-06-21 09:30:00.205075239    10
 2012-06-21 09:30:00.205158699     9
 2012-06-21 09:30:02.190134797     4
 2012-06-21 09:30:02.190251279     6
 2012-06-21 09:30:02.203099603     4
                                  ..
 2012-06-21 15:59:58.401139334     5
 2012-06-21 15:59:58.836501474     7
 2012-06-21 15:59:59.014127805     4
 2012-06-21 15:59:59.458113092     4
 2012-06-21 15:59:59.582154048     6
 Length: 31743, dtype: int64)

In [569]:
cnl_lvl_ask, cnl_lvl_bid = eval.cancel_order_levels(m_real, b_real)

In [571]:
eval.mean_per_interval(cnl_lvl_ask, "5min")

time
2012-06-21 09:35:00    2.160839
2012-06-21 09:40:00    3.455939
2012-06-21 09:45:00    4.217573
2012-06-21 09:50:00    3.305361
2012-06-21 09:55:00    3.300172
                         ...   
2012-06-21 15:40:00    4.931330
2012-06-21 15:45:00    4.398458
2012-06-21 15:50:00    3.661538
2012-06-21 15:55:00    3.778420
2012-06-21 16:00:00    3.426070
Freq: 5min, Length: 78, dtype: float64

In [ ]:
# provide loader fn: takes index -> returns date, real messages, real book, gen messages, (gen book)

# 
# 

In [604]:
from data_loading import Simple_Loader, Lobster_Sequence, Lazy_Tuple

In [609]:
ls = Lobster_Sequence(
    "2024-01-01",
    lambda: 1,
    lambda: 2,
    (lambda: 3, lambda: 4),
    (lambda: 5, lambda: 6),
    2
)

In [612]:
tuple(ls.m_gen)

(3, 4)

In [853]:
loader = data_loading.Simple_Loader('data_test_real', 'data_test_gen', 'data_test_cond')
s = loader[0]
# s.materialize()

In [857]:
s.m_real

,time,event_type,order_id,size,price,direction
0,2012-06-21 09:30:00.015105074,4,15818974,4,5794000,1
1,2012-06-21 09:30:00.059901970,4,15818974,300,5794000,1
2,2012-06-21 09:30:00.113246707,5,0,1,5795100,1
3,2012-06-21 09:30:00.113246707,5,0,1,5795000,1
4,2012-06-21 09:30:00.113246707,5,0,1,5794900,1
...,...,...,...,...,...,...
147911,2012-06-21 15:59:59.544131783,1,287157290,300,5655600,-1
147912,2012-06-21 15:59:59.544132627,1,287157291,200,5655600,-1
147913,2012-06-21 15:59:59.544133264,1,287157292,300,5655600,-1
147914,2012-06-21 15:59:59.582154048,3,287027555,100,5649700,1


In [862]:
partitioning.score_real(loader, lambda x, y: 1)

array([1])

In [868]:
scores = partitioning.score_real(
    tuple(partitioning.get_subseqs(s, time_interval='10min') for s in loader),
    lambda m, b: m.time.diff().mean()
)

In [872]:
len(scores)

1

In [875]:
partitioning.group_by_score(scores)

[Timedelta('0 days 00:00:00.098292210'), Timedelta('0 days 00:00:00.217133571'), Timedelta('0 days 00:00:00.158343273'), Timedelta('0 days 00:00:00.090068273'), Timedelta('0 days 00:00:00.125035999'), Timedelta('0 days 00:00:00.138976549'), Timedelta('0 days 00:00:00.123208587'), Timedelta('0 days 00:00:00.112410261'), Timedelta('0 days 00:00:00.156428270'), Timedelta('0 days 00:00:00.163456348'), Timedelta('0 days 00:00:00.113367037'), Timedelta('0 days 00:00:00.203282383'), Timedelta('0 days 00:00:00.295049385'), Timedelta('0 days 00:00:00.162072288'), Timedelta('0 days 00:00:00.199465790'), Timedelta('0 days 00:00:00.204993594'), Timedelta('0 days 00:00:00.246893592'), Timedelta('0 days 00:00:00.160483283'), Timedelta('0 days 00:00:00.182061350'), Timedelta('0 days 00:00:00.169065219'), Timedelta('0 days 00:00:00.297406124'), Timedelta('0 days 00:00:00.273141397'), Timedelta('0 days 00:00:00.217005387'), Timedelta('0 days 00:00:00.183154494'), Timedelta('0 days 00:00:00.147465929'),

In [851]:
s.b_cond

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,5802300,100,5794000,496,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
1,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
2,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
3,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
4,5802300,100,5794000,196,5804300,100,5787000,400,5805000,100,...,5762600,100,5880000,100,5732000,20,5892600,100,5700000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147911,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2200,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147912,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2400,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147913,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100
147914,5652200,42,5651200,200,5655000,200,5651100,100,5655600,2700,...,5649400,100,5657000,1800,5649200,300,5657500,100,5649100,100


In [849]:
subseq = partitioning.get_subseqs(s, subseq_len=1000)

/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/opt/miniconda3/envs/ml/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [817]:
len(subseq.m_real)

148

In [766]:
s.m_gen[0]

,time,event_type,order_id,size,price,direction
0,2012-06-21 09:30:00.015105074,4,15818974,4,5794000,1
1,2012-06-21 09:30:00.059901970,4,15818974,300,5794000,1
2,2012-06-21 09:30:00.113246707,5,0,1,5795100,1
3,2012-06-21 09:30:00.113246707,5,0,1,5795000,1
4,2012-06-21 09:30:00.113246707,5,0,1,5794900,1
...,...,...,...,...,...,...
147911,2012-06-21 15:59:59.544131783,1,287157290,300,5655600,-1
147912,2012-06-21 15:59:59.544132627,1,287157291,200,5655600,-1
147913,2012-06-21 15:59:59.544133264,1,287157292,300,5655600,-1
147914,2012-06-21 15:59:59.582154048,3,287027555,100,5649700,1


In [728]:
import partitioning

m_real, b_real, m_gen, b_gen = partitioning.get_subseqs(loader[0], time_interval='5min')

In [733]:
m_real()

(                              time  event_type  order_id  size    price  \
 0    2012-06-21 09:30:00.015105074           4  15818974     4  5794000   
 1    2012-06-21 09:30:00.059901970           4  15818974   300  5794000   
 2    2012-06-21 09:30:00.113246707           5         0     1  5795100   
 3    2012-06-21 09:30:00.113246707           5         0     1  5795000   
 4    2012-06-21 09:30:00.113246707           5         0     1  5794900   
 ...                            ...         ...       ...   ...      ...   
 1732 2012-06-21 09:34:55.668315019           3  23158746   100  5786300   
 1733 2012-06-21 09:34:55.668315019           1  23165724   100  5786500   
 1734 2012-06-21 09:34:55.675547545           3  23165724   100  5786500   
 1735 2012-06-21 09:34:55.675547545           1  23166153   100  5786700   
 1736 2012-06-21 09:34:59.008617033           1  23207675   100  5772100   
 
       direction  
 0             1  
 1             1  
 2             1  
 3        

In [646]:
pd.concat([m_real, b_real], axis=1).groupby(
    pd.Grouper(key='time', freq='5s')
)